In [2]:
%matplotlib notebook
%load_ext autoreload
%pwd

'/ocean/projects/asc170022p/mtragoza/lung-project/notebooks'

In [4]:
import sys
sys.path.append('../../param_search')
import param_search as ps

In [13]:
# define a job template and name format
template = '''\
#!/bin/bash
#SBATCH --job-name={job_name}
#SBATCH --account=asc170022p
#SBATCH --partition=GPU-shared
#SBATCH --gres=gpu:1
#SBATCH -x v034
#SBATCH --time=48:00:00
#SBATCH -o %J.stdout
#SBATCH -e %J.stderr
#SBATCH --mail-type=all

hostname
pwd
module load anaconda3
conda activate /ocean/projects/asc170022p/mtragoza/mambaforge/envs/lung-project
nvidia-smi

data_root=../../../data/{data_name}

case_name=Case{case_id:d}Pack
case_dir=$data_root/$case_name

image_dir=$case_dir/NIFTI
mask_dir=$case_dir/TotalSegment

image_name=case{case_id:d}_T{phase:02d}
image_file=$image_dir/$image_name.nii.gz
output_dir=$mask_dir/$image_name

TotalSegmentator -i $image_file -o $output_dir -d gpu -ta total --roi_subset lung_upper_lobe_right lung_upper_lobe_left lung_middle_lobe_right lung_lower_lobe_right lung_lower_lobe_left
TotalSegmentator -i $image_file -o $output_dir -d gpu -ta lung_vessels
TotalSegmentator -i $image_file -o $output_dir -d gpu -ta body
totalseg_combine_masks -i $output_dir -o $output_dir/lung_combined_mask.nii.gz -m lung

echo Done
'''
name_format = 'segment_{data_name}_case{case_id:d}_T{phase:02d}'

In [14]:
param_space = ps.ParamSpace(
    data_name='Emory-4DCT',
    case_id=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    phase=[0, 10, 20, 30, 40, 50, 60, 70, 80, 90]
)

for p in param_space:
    print(name_format.format(**p))

segment_Emory-4DCT_case1_T00
segment_Emory-4DCT_case1_T10
segment_Emory-4DCT_case1_T20
segment_Emory-4DCT_case1_T30
segment_Emory-4DCT_case1_T40
segment_Emory-4DCT_case1_T50
segment_Emory-4DCT_case1_T60
segment_Emory-4DCT_case1_T70
segment_Emory-4DCT_case1_T80
segment_Emory-4DCT_case1_T90
segment_Emory-4DCT_case2_T00
segment_Emory-4DCT_case2_T10
segment_Emory-4DCT_case2_T20
segment_Emory-4DCT_case2_T30
segment_Emory-4DCT_case2_T40
segment_Emory-4DCT_case2_T50
segment_Emory-4DCT_case2_T60
segment_Emory-4DCT_case2_T70
segment_Emory-4DCT_case2_T80
segment_Emory-4DCT_case2_T90
segment_Emory-4DCT_case3_T00
segment_Emory-4DCT_case3_T10
segment_Emory-4DCT_case3_T20
segment_Emory-4DCT_case3_T30
segment_Emory-4DCT_case3_T40
segment_Emory-4DCT_case3_T50
segment_Emory-4DCT_case3_T60
segment_Emory-4DCT_case3_T70
segment_Emory-4DCT_case3_T80
segment_Emory-4DCT_case3_T90
segment_Emory-4DCT_case4_T00
segment_Emory-4DCT_case4_T10
segment_Emory-4DCT_case4_T20
segment_Emory-4DCT_case4_T30
segment_Emory-

In [15]:
jobs = ps.submit(template, name_format, param_space, work_dir='2024-05-13_segment')
jobs

100%|██████████| 100/100 [00:07<00:00, 13.86it/s]


/ocean/projects/asc170022p/mtragoza/lung-project/notebooks/../../param_search/param_search/job_queues.py:235: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace('', float('nan')).map(pd.to_numeric)


,data_name,case_id,phase,job_name,job_id,partition,job_state,node_id,runtime,work_dir,array_idx
0,Emory-4DCT,1,0,segment_Emory-4DCT_case1_T00,23842490,GPU-shared,PENDING,(Priority),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN
1,Emory-4DCT,1,10,segment_Emory-4DCT_case1_T10,23842491,GPU-shared,PENDING,(Priority),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN
2,Emory-4DCT,1,20,segment_Emory-4DCT_case1_T20,23842492,GPU-shared,PENDING,(Priority),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN
3,Emory-4DCT,1,30,segment_Emory-4DCT_case1_T30,23842493,GPU-shared,PENDING,(Priority),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN
4,Emory-4DCT,1,40,segment_Emory-4DCT_case1_T40,23842494,GPU-shared,PENDING,(Priority),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN
...,...,...,...,...,...,...,...,...,...,...,...
95,Emory-4DCT,10,50,segment_Emory-4DCT_case10_T50,23842586,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN
96,Emory-4DCT,10,60,segment_Emory-4DCT_case10_T60,23842587,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN
97,Emory-4DCT,10,70,segment_Emory-4DCT_case10_T70,23842588,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN
98,Emory-4DCT,10,80,segment_Emory-4DCT_case10_T80,23842589,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN


In [17]:
status = ps.status(jobs)
status

,index,data_name,case_id,phase,job_name,partition,job_state,node_id,runtime,work_dir,array_idx,stdout,stderr
job_id,,,,,,,,,,,,,
23842490,0,Emory-4DCT,1,0,segment_Emory-4DCT_case1_T00,GPU-shared,NaN,NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,v013.ib.bridges2.psc.edu\n/ocean/projects/asc1...,/ocean/projects/asc170022p/mtragoza/mambaforge...
23842491,1,Emory-4DCT,1,10,segment_Emory-4DCT_case1_T10,GPU-shared,NaN,NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,v013.ib.bridges2.psc.edu\n/ocean/projects/asc1...,/ocean/projects/asc170022p/mtragoza/mambaforge...
23842492,2,Emory-4DCT,1,20,segment_Emory-4DCT_case1_T20,GPU-shared,NaN,NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,v013.ib.bridges2.psc.edu\n/ocean/projects/asc1...,/ocean/projects/asc170022p/mtragoza/mambaforge...
23842493,3,Emory-4DCT,1,30,segment_Emory-4DCT_case1_T30,GPU-shared,NaN,NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,v029.ib.bridges2.psc.edu\n/ocean/projects/asc1...,/ocean/projects/asc170022p/mtragoza/mambaforge...
23842494,4,Emory-4DCT,1,40,segment_Emory-4DCT_case1_T40,GPU-shared,NaN,NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,v029.ib.bridges2.psc.edu\n/ocean/projects/asc1...,/ocean/projects/asc170022p/mtragoza/mambaforge...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23842586,95,Emory-4DCT,10,50,segment_Emory-4DCT_case10_T50,GPU-shared,NaN,NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,v022.ib.bridges2.psc.edu\n/ocean/projects/asc1...,/ocean/projects/asc170022p/mtragoza/mambaforge...
23842587,96,Emory-4DCT,10,60,segment_Emory-4DCT_case10_T60,GPU-shared,NaN,NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,v022.ib.bridges2.psc.edu\n/ocean/projects/asc1...,/ocean/projects/asc170022p/mtragoza/mambaforge...
23842588,97,Emory-4DCT,10,70,segment_Emory-4DCT_case10_T70,GPU-shared,NaN,NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,v022.ib.bridges2.psc.edu\n/ocean/projects/asc1...,/ocean/projects/asc170022p/mtragoza/mambaforge...


In [18]:
print(status.stdout.iloc[0])

v013.ib.bridges2.psc.edu
/ocean/projects/asc170022p/mtragoza/lung-project/notebooks/2024-05-13_segment/segment_Emory-4DCT_case1_T00
Tue May 14 00:10:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           On  | 00000000:B2:00.0 Off |                    0 |
| N/A   27C    P0              41W / 300W |      0MiB / 32768MiB |      0%      Default |
|                         

In [19]:
print(status.stderr.iloc[0])

/ocean/projects/asc170022p/mtragoza/mambaforge/envs/lung-project/lib/python3.10/site-packages/nnunetv2/utilities/plans_handling/plans_handler.py:37: UserWarning: Detected old nnU-Net plans format. Attempting to reconstruct network architecture parameters. If this fails, rerun nnUNetv2_plan_experiment for your dataset. If you use a custom architecture, please downgrade nnU-Net to the version you implemented this or update your implementation + plans.
  warnings.warn("Detected old nnU-Net plans format. Attempting to reconstruct network architecture "

  0%|          | 0/1 [00:00<?, ?it/s]/ocean/projects/asc170022p/mtragoza/mambaforge/envs/lung-project/lib/python3.10/site-packages/torch/nn/modules/conv.py:605: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1712608839953/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:84.)
  return F.conv3d(

100%|██████████| 1/1 [00:12<00:00, 12.49s/it]
/ocean/projects/asc170022p/mtragoz

In [22]:
import sys, os
os.environ['PKG_CONFIG_PATH'] = '/ocean/projects/asc170022p/mtragoza/mambaforge/envs/lung-project/lib/pkgconfig'
sys.path.append('..')
import mycode as code
import hvplot.xarray

AttributeError: module 'fenics' has no attribute 'cpp'

In [ ]:
%autoreload
case = code.data.Emory4DCTCase('data/Emory-4DCT', 'Case1Pack', [0, 50])
case.load_niftis()
case.load_masks()

In [ ]:
data.view_array(case.array, groupby=['phase', 'y'], x='x', y='z')

In [ ]:
case.describe()

In [ ]:
import vedo
vedo.settings.default_backend= 'ipyvtklink'
import visual

In [ ]:
%autoreload

visual.IsosurfaceBrowser(
    ds.array,
    isovalue=845,
    c='white',
    alpha=0.5,
    use_gpu=True
)

In [ ]:
help(vedo.Volume)